# Fine-Tuned Bert Model For Text Classification with LIME Analysis

In [1]:
# Whole dataset
! gdown 1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01 # all training data
! gdown 1FhT3m_ApKzX615JzshB5-d-j6S91-6oz # all validation data
! gdown 16p0td9GgJRb9AP8i4HlX-xZGI2u849uA # all testing data


# https://drive.google.com/file/d/1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01/view?usp=sharing

! mkdir open_llm
! mv train.jsonl valid.jsonl test.jsonl open_llm/

! pip install datasets

Downloading...
From (original): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01
From (redirected): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01&confirm=t&uuid=c726ecaa-8f9a-4b20-b4e4-7cb299501288
To: /content/train.jsonl
100% 292M/292M [00:03<00:00, 86.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FhT3m_ApKzX615JzshB5-d-j6S91-6oz
To: /content/valid.jsonl
100% 55.1M/55.1M [00:00<00:00, 97.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=16p0td9GgJRb9AP8i4HlX-xZGI2u849uA
To: /content/test.jsonl
100% 39.1M/39.1M [00:00<00:00, 179MB/s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.4 MB/s eta 0:00

In [2]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=46590f47296d208d24e3dcfe50892dc20e0d8ca0baedd7d8f59009db91da2bed
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [3]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import Dataset
import pandas as pd
import re
import lime
import numpy as np

In [4]:
pretrained_model_name = "bert-base-uncased"
model = TFAutoModel.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
df_train = pd.read_json("open_llm/train.jsonl", lines=True)
df_valid = pd.read_json("open_llm/valid.jsonl", lines=True)
df_test = pd.read_json("open_llm/test.jsonl", lines=True)

df_train = df_train.drop(columns=['extra', 'source'])
df_valid = df_valid.drop(columns=['extra', 'source'])
df_test = df_test.drop(columns=['extra', 'source'])

df_train = df_train[:20000]
df_test = df_test[:3000]
df_valid = df_valid[:1000]

In [7]:
df_train

,uid,text,label
0,[urlsf_subset00]-[15],The dangers of Illinois as a ‘right to work’ s...,0
1,[urlsf_subset00]-[15],"The governor of Illinois, Gov. Rauner, has req...",1
2,[urlsf_subset00]-[83],Check current weather conditions\n\nIt’s going...,0
3,[urlsf_subset00]-[83],Check current weather conditions It’s going to...,1
4,[urlsf_subset00]-[89],"On Thursday, the president of the United State...",0
...,...,...,...
19995,[urlsf_subset01]-[106131],"On Tuesday, the Senate Intelligence Committee ...",1
19996,[urlsf_subset01]-[106225],"WEST DES MOINES, Iowa — A West Des Moines Cath...",0
19997,[urlsf_subset01]-[106225],"WEST DES MOINES, Iowa — A West Des Moines Cath...",1
19998,[urlsf_subset01]-[106271],The Advertising Standards Authority has slappe...,0


In [8]:
def text_process(mess):
    """
    Process text to:
    1. Remove punctuation (including all Unicode quotes)
    2. Convert text to lowercase
    3. Return cleaned text without removing stopwords
    """
    mess = re.sub(r"[^\w\s]", "", mess)

    mess = mess.lower()

    return mess

In [9]:
df_train['text'] = df_train['text'].apply(text_process)
df_valid['text'] = df_valid['text'].apply(text_process)
df_test['text'] = df_test['text'].apply(text_process)

In [10]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

In [11]:
# Convert pandas dataframes to Hugging Face datasets
hf_train = Dataset.from_pandas(df_train)
hf_valid = Dataset.from_pandas(df_valid)
hf_test = Dataset.from_pandas(df_test)

# Apply the tokenization
hf_train_encoded = hf_train.map(tokenize, batched=True)
hf_valid_encoded = hf_valid.map(tokenize, batched=True)
hf_test_encoded = hf_test.map(tokenize, batched=True)

# Preview the results
print(hf_train_encoded)
print(hf_valid_encoded)
print(hf_test_encoded)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})
Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})
Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3000
})


In [12]:
hf_train_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
hf_valid_encoded.set_format(
    type='tf',
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

BATCH_SIZE = 16

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(hf_train_encoded[:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# same for test set
test_dataset = tf.data.Dataset.from_tensor_slices(hf_test_encoded[:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [13]:
valid_dataset = tf.data.Dataset.from_tensor_slices(hf_valid_encoded[:])
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [14]:
inp, out = next(iter(train_dataset)) # one batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[  101,  2023,  3134, ...,  1015,  8875,   102],
       [  101,  2023,  3134, ...,     0,     0,     0],
       [  101,  2420,  2044, ...,     0,     0,     0],
       ...,
       [  101,  2019, 12021, ...,     0,     0,     0],
       [  101,  2043,  2009, ...,     0,     0,     0],
       [  101,  2043,  2009, ...,  8569,  9215,   102]])>, 'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])>} 

 tf.Tensor([0 1 0 1 0 1 0 1 0 1 0 1 

In [15]:
inp, out = next(iter(valid_dataset)) # one batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[  101,  9317,  2258, ...,  2278,  2581,   102],
       [  101,  1037,  3232, ...,     0,     0,     0],
       [  101,  2624,  3799, ...,     0,     0,     0],
       ...,
       [  101,  5639, 18090, ...,     0,     0,     0],
       [  101,  1037, 14487, ...,  2000,  2022,   102],
       [  101,  1037, 14487, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([0 1 0 1 0 1 0 1 0 1 0 1 

In [16]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, model, num_classes, dropout=0.3):
        super(BERTForClassification, self).__init__()
        self.bert = model
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        outputs = self.bert(**inputs)
        pooled_output = outputs[1]  # CLS token embedding
        x = self.dropout(pooled_output, training=training)
        return self.classifier(x)

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
classifier = BERTForClassification(model, num_classes=2, dropout=0.5)

classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [18]:
history = classifier.fit(
    train_dataset,
    epochs=3,
    validation_data=valid_dataset
)

Epoch 1/3
1250/1250 [==============================] - 332s 231ms/step - loss: 0.3569 - accuracy: 0.8354 - val_loss: 0.1864 - val_accuracy: 0.9280
Epoch 2/3
1250/1250 [==============================] - 285s 228ms/step - loss: 0.1428 - accuracy: 0.9447 - val_loss: 0.1971 - val_accuracy: 0.9180
Epoch 3/3
1250/1250 [==============================] - 285s 228ms/step - loss: 0.0841 - accuracy: 0.9694 - val_loss: 0.2496 - val_accuracy: 0.9140


In [19]:
# save current state of classifier
classifier.save_weights('classifier_weights_v2.h5')

In [21]:
from lime.lime_text import LimeTextExplainer


def lime_predict(texts):
    """
    Predict probabilities for a list of texts using the BERT classifier.
    """
    # Batch predictions to avoid memory issues
    batch_size = 32
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        encoded_inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=256,  # Limit to 256 tokens to reduce memory usage
            return_tensors="tf"
        )
        # Get predictions from the model
        predictions = classifier(encoded_inputs, training=False)
        results.append(predictions.numpy())
    return np.vstack(results)

explainer = LimeTextExplainer(class_names=["Human Text", "AI Text"])

# Select instances from the test set with the 8th and 9th index
example_texts = [
    str(df_test['text'].iloc[8]),
    str(df_test['text'].iloc[9])
]

# Generate explanations for the 9th and 10th texts
for i, text in enumerate(example_texts, start=9):
    explanation = explainer.explain_instance(
        text_instance=text,
        classifier_fn=lime_predict,
        num_features=12, # Show 12 top features
        num_samples=2000 # Reduce number of perturbations
    )

    # Save the explanation to an HTML file for visualization
    explanation.save_to_file(f"lime_explanation_text_{i}.html")

    # Display the explanation in the notebook
    explanation.show_in_notebook()


Output hidden; open in https://colab.research.google.com to view.